In [12]:
import pandas as pd
import re
from datetime import datetime
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# produce word counts
def wordCount(df, label='All', time=24):
  if time != 24:
    if time in df['hour'].unique():
      df = df[df['hour']==time]
    else:
      return []
  if label != 'All':
    df = df[df[label.lower()]==1]
  word_all = {}
  for index, row in df.iterrows():
    for word in row['token']:
      if word not in word_all.keys():
        word_all[word] = 1
      else:
        word_all[word] += 1

  words = []
  for key in word_all.keys():
    words.append({"text": key, "size": word_all[key]})
  return words, len(df)

In [26]:
df = pd.read_csv("tweets_22sample.csv")
df['date'] = pd.to_datetime(df['created_at'], format='%Y-%m-%d %H:%M:%S')
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df_eng = df[df['lang']=='en']
df_eng['hour'].unique()

array([15, 21, 18, 16, 20, 19, 22, 17, 23])

In [24]:
# Clean the tokens, remove punctuations and stop words
token = [doc.split() for doc in df_eng['text']]
token = [[word.lower() for word in word_list if word.lower() not in stopwords.words('english') and not re.search(r'RT.*|@.*|https://.*', word)] for word_list in token]
token = [[re.sub(r'[^\w\s]','',word) for word in word_list] for word_list in token]
# add tokens to dataframe
df_token = df_eng
df_token['token'] = token
df_token = df_token.drop(columns = ['text'])

# identify tweets mentioning the two candidates
df_label = df_token
df_label['trump'] = -1
df_label['biden'] = -1
for index, row in df_label.iterrows():
    df_label.loc[index, ['trump']] = 1 if any(word in ['trump', 'donald', 'donaldtrump'] for word in row['token']) else 0
    df_label.loc[index, ['biden']] = 1 if any(word in ['biden', 'joe', 'joebiden'] for word in row['token']) else 0
df_label.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,created_at,geo,lang,place,coordinates,user.favourites_count,user.statuses_count,user.description,user.location,user.id,user.created_at,user.verified,user.following,user.url,user.listed_count,user.followers_count,user.default_profile_image,user.utc_offset,user.friends_count,user.default_profile,user.name,user.lang,user.screen_name,user.geo_enabled,user.profile_background_color,user.profile_image_url,user.time_zone,id,favorite_count,retweeted,source,favorited,retweet_count,date,day,hour,token,trump,biden
0,2020-10-22 15:40:49+00:00,NaN,en,NaN,NaN,2818,6784,Independent - Need new Real Conservative Party...,Utah,63038774,2009-08-05 05:02:28+00:00,False,False,NaN,14,399,False,NaN,827,False,Cathy Richardson,NaN,catpaq,True,BADFCD,http://pbs.twimg.com/profile_images/1323650349...,NaN,1319302699460878337,0,False,Twitter Web App,False,0,2020-10-22 15:40:49+00:00,22,15,"[option, rude, outright, telling, lies, go, tr...",1,0
2,2020-10-22 21:41:43+00:00,NaN,en,NaN,NaN,19680,51756,Happily retired reading teacher and enjoying m...,NaN,3178461490,2015-04-18 01:49:30+00:00,False,False,NaN,3,522,False,NaN,2820,True,KBMOF,NaN,KBMOF,False,C0DEED,http://pbs.twimg.com/profile_images/1327810173...,NaN,1319393523804835842,0,False,Twitter for iPhone,False,794,2020-10-22 21:41:43+00:00,22,21,"[legendary, awardwinning, journalist, connie, ...",0,0
3,2020-10-22 21:41:03+00:00,NaN,en,NaN,NaN,74016,106175,Retired. Liberal. Into American politics when ...,"Ontario, Canada 🇨🇦",1216746750918131714,2020-01-13 15:40:18+00:00,False,False,NaN,21,5076,False,NaN,3920,True,Diane Crawford 🇨🇦,NaN,retiredknitwit,False,F5F8FA,http://pbs.twimg.com/profile_images/1459162488...,NaN,1319393355843907584,0,False,Twitter for iPhone,False,0,2020-10-22 21:41:03+00:00,22,21,"[trump, lowered, corporate, tax, rate, 21, ceo...",1,0
4,2020-10-22 18:55:16+00:00,NaN,en,NaN,NaN,104130,39498,Person of Interest🔞 | Producer 🎥🎞 | Industry R...,in your dreams☁️☁️☁️,904393668,2012-10-25 18:15:02+00:00,False,False,https://t.co/sKg1WZZ4iu,203,67632,False,NaN,2597,False,✨BLM forever✊🏾,NaN,itsLotusLain,True,EBEBEB,http://pbs.twimg.com/profile_images/1260733510...,NaN,1319351634611171328,0,False,Twitter for iPhone,False,1,2020-10-22 18:55:16+00:00,22,18,"[sent, ballot, last, week, hbu]",0,0
5,2020-10-22 15:46:18+00:00,NaN,en,NaN,NaN,1943,891,"Sometimes I write code, sometimes I make UI @ ...","Toronto, Ontario",3386277568,2015-07-21 19:07:32+00:00,False,False,https://t.co/Gg1ejArKAA,1,118,False,NaN,154,False,Michael Too Many Keyboards Nicoletti,NaN,GuyWifGlasses,True,000000,http://pbs.twimg.com/profile_images/1438693000...,NaN,1319304080016216064,0,False,Twitter Web App,False,2453,2020-10-22 15:46:18+00:00,22,15,"[what, dutch, researcher, managed, log, twitte...",0,0


In [27]:
wordcloud = pd.DataFrame(columns = ["Date", "Time", "Candidate", "Words", "Tweets"])
df = df_label
for i in range(15, 25):
  candidates = ["All", "Biden", "Trump"]
  for candidate in candidates:
    word, tweets = wordCount(df, candidate, i)
    newRow = {'Date': 22, 'Time': i,'Candidate': candidate, "Words": word, "Tweets": tweets}
    wordcloud = wordcloud.append(newRow, ignore_index=True)
wordcloud

,Date,Time,Candidate,Words,Tweets
0,22,15,All,"[{'text': 'option', 'size': 1}, {'text': 'rude...",240
1,22,15,Biden,"[{'text': 'scandal', 'size': 3}, {'text': 'rep...",34
2,22,15,Trump,"[{'text': 'option', 'size': 1}, {'text': 'rude...",55
3,22,16,All,"[{'text': 'see', 'size': 12}, {'text': '2', 's...",673
4,22,16,Biden,"[{'text': 'stahl', 'size': 4}, {'text': 'notes...",106
5,22,16,Trump,"[{'text': 'see', 'size': 6}, {'text': '2', 'si...",229
6,22,17,All,"[{'text': 'republicans', 'size': 13}, {'text':...",732
7,22,17,Biden,"[{'text': 'david', 'size': 1}, {'text': 'marcu...",125
8,22,17,Trump,"[{'text': 'saw', 'size': 3}, {'text': 'greater...",222
9,22,18,All,"[{'text': 'sent', 'size': 4}, {'text': 'ballot...",634


In [31]:
df = pd.read_csv("tweets_23sample.csv")
df['date'] = pd.to_datetime(df['created_at'], format='%Y-%m-%d %H:%M:%S')
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df_eng = df[df['lang']=='en']

# Clean the tokens, remove punctuations and stop words
token = [doc.split() for doc in df_eng['text']]
token = [[word.lower() for word in word_list if word.lower() not in stopwords.words('english') and not re.search(r'RT.*|@.*|https://.*', word)] for word_list in token]
token = [[re.sub(r'[^\w\s]','',word) for word in word_list] for word_list in token]
df_token = df_eng
df_token['token'] = token
df_token = df_token.drop(columns = ['text'])
# identify tweets mentioning the two 
df_label = df_token
df_label['trump'] = -1
df_label['biden'] = -1
for index, row in df_label.iterrows():
  # if any(word in ['trump', 'donald', 'donaldtrump'] for word in row['token']):
    df_label.loc[index, ['trump']] = 1 if any(word in ['trump', 'donald', 'donaldtrump'] for word in row['token']) else 0
    df_label.loc[index, ['biden']] = 1 if any(word in ['biden', 'joe', 'joebiden'] for word in row['token']) else 0
df = df_label
for i in range(0,25):
  candidates = ["All", "Biden", "Trump"]
  for candidate in candidates:
    word, tweets = wordCount(df, candidate, i)
    newRow = {'Date': 23, 'Time': i,'Candidate': candidate, "Words": word, "Tweets": tweets}
    wordcloud = wordcloud.append(newRow, ignore_index=True)
wordcloud

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Date,Time,Candidate,Words,Tweets
0,22,15,All,"[{'text': 'option', 'size': 1}, {'text': 'rude...",240
1,22,15,Biden,"[{'text': 'scandal', 'size': 3}, {'text': 'rep...",34
2,22,15,Trump,"[{'text': 'option', 'size': 1}, {'text': 'rude...",55
3,22,16,All,"[{'text': 'see', 'size': 12}, {'text': '2', 's...",673
4,22,16,Biden,"[{'text': 'stahl', 'size': 4}, {'text': 'notes...",106
...,...,...,...,...,...
100,23,23,Biden,"[{'text': 'word', 'size': 1}, {'text': 'whethe...",48
101,23,23,Trump,"[{'text': 'actually', 'size': 2}, {'text': 'di...",74
102,23,24,All,"[{'text': 'unbelievable', 'size': 12}, {'text'...",5780
103,23,24,Biden,"[{'text': 'president', 'size': 131}, {'text': ...",1341


In [33]:
wordcloud.to_csv("tweets_statistics.csv")